## Migrador instalaciones régimen especial

In [162]:
import pandas as pd
from erppeek import Client
from datetime import datetime

# conexión
try:
    # Due: https://www.python.org/dev/peps/pep-0476
    import ssl
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        # Legacy Python that doesn't verify HTTPS certificates by default
        pass
    else:
        # Handle target environment that doesn't support HTTPS verification
        ssl._create_default_https_context = _create_unverified_https_context
except ImportError:
    pass

from tqdm import tqdm

c = Client('http://localhost:8069', 'test_1571740412', 'admin', 'admin')
cc = Client('http://ecasa-erp.clients:28069', 'PRE_ecasa_distri', 'gisce')
migrator(cc, 'UPRSL_0034_20191001.13', 'UFIUPR_0034_20191001.13', 'CILDATOS_0034_20191018.0', 'CA')

  0%|          | 0/15 [00:00<?, ?it/s]

Titular creat: SOCIEDAD GRAL BAHIAS DE CADIZ Y ALGECIRAS, SA


  7%|▋         | 1/15 [00:00<00:11,  1.22it/s]

Representante creado. Validar datos: NEXUS ENERGIA, S.A
Titular creat: COLEGIO OFICIAL INGL INDS. DE ANDALUCIA OCCIDENTAL


 13%|█▎        | 2/15 [00:01<00:09,  1.34it/s]

...

 87%|████████▋ | 13/15 [00:06<00:00,  2.13it/s]

Titular creat: GESTORA GADITANA S.A


 93%|█████████▎| 14/15 [00:06<00:00,  2.15it/s]

Titular creat: PLANTA GENERADORA FOTOVOLTAICA EN CUBIERTA, UCAFOTON S.L


100%|██████████| 15/15 [00:07<00:00,  2.03it/s]


In [158]:
def migrator(c, uprsl_name=None, ufiupr_name=None, cildatos_name=None, provincia=None):
    # objects
    partner_obj = c.model('res.partner')
    partner_addr = c.model('res.partner.address')
    instalacion_re_obj = c.model('giscedata.re')
    unidad_programacion_obj = c.model('giscedata.re.uprs')
    cups_obj = c.model('giscedata.cups.ps')
    municipi_obj = c.model('res.municipi')

    # df unidades de programación
    df_unidades_de_programacion = pd.read_csv(uprsl_name, 
                      sep = ';',
                     names = ['codigo', 
                            'CIF', 
                            'representante', 
                            'rel_upr_2', 
                            'name_upr', 
                            'desc_upr', 
                            'cod_participante', 
                            'tarifa_o_mercado',
                            'tecnologia', 
                            'exenta_pago', 
                            'data_alta',
                            'data_baixa', 
                            '_'], dtype={'data_alta': str, 'data_baixa': str})
    
    df_unidades_de_programacion.data_alta = df_unidades_de_programacion.data_alta.apply(lambda x: '{0}-{1}-{2} 00:00:00'.format(x[:4], x[4:6], x[6:8]))
    df_unidades_de_programacion['data_baixa'] = df_unidades_de_programacion.data_baixa.apply(lambda x: '{0}-{1}-{2} 00:00:00'.format(x[:4], x[4:6], x[6:8]))
    
    df_unidades_de_programacion['cod_participante'] = df_unidades_de_programacion['cod_participante'].apply(
        lambda x: str(x).zfill(4))

    # df instalaciones re
    df_instalaciones_re = pd.read_csv(ufiupr_name, 
                     sep = ';', 
                     names = ['codigo_unidad_fisica', 
                            'name_upr', 
                            'name',
                            'descripcio', 
                            'raipre', 
                            'cups', 
                            'cups2', 
                            'data_inici', 
                            'data_final', 
                            '_'])

    # df cildatos (tipo_punto_frontera)
    df_cildatos = pd.read_csv(cildatos_name, 
                     sep = ';',
                     names = ['cups', 
                            'minetad', 
                            'num_reg_provisional', 
                            'num_reg_definitiva', 
                            'cod_postal', 
                            'potencia_nominal', 
                            'nombre_instalacion', 
                            'psse',
                            'grupo_rd', 
                            'tipo_punto_fontera', 
                            'fecha_alta',
                            'fecha_baja',
                            'tensión_conexión',
                            'fecha_acta_alta',
                            'propiedad_equipo_medida',
                            '_'],
                             dtype={'tipo_punto_fontera': str, 'potencia_nominal': float})
    df_cildatos['cups'] = df_cildatos['cups'].apply(lambda x: x[:-3])

    # merge dfs
    df_instalaciones_re = pd.merge(df_unidades_de_programacion, df_instalaciones_re, on='name_upr', how='right')
    df_instalaciones_re = pd.merge(df_cildatos, df_instalaciones_re, on='cups', how='right')
    df_instalaciones_re = df_instalaciones_re[['CIF', 
                                               'representante', 
                                               'name_upr', 
                                               'cod_participante', 
                                               'cups', 
                                               'data_alta', 
                                               'data_baixa', 
                                               'name',
                                               'tipo_punto_fontera',
                                               'potencia_nominal',]]

    # TODO: DEFAULT instalaciones re
    df_instalaciones_re['provincia'] = provincia

    # migrar datos
    for instalacion_re_data in tqdm(df_instalaciones_re.T.to_dict().values()):

        # buscar titular para la instalacion re
        titular_name = instalacion_re_data.get('name')
        titular_ids = partner_obj.search([('name', '=', titular_name)])
        if not titular_ids:
            # crear titular
            titular_id = partner_obj.create({'name': titular_name, 'vat': ''})
            titular_id = titular_id.id
            p_address_id = partner_addr.create({'name': titular_name, 'partner_id': titular_id})
            print('Titular creat: {}'.format(titular_name))
        else:
            titular_id = titular_ids[0]

        # buscar cups para la instalacion re
        cups_name = instalacion_re_data.get('cups')
        cups_ids = cups_obj.search([('name', '=', cups_name)])
        if not cups_ids:
            # crear cups
            # municipi_ids = municipi_obj.search([('name', '=', municipi_name)])
            # cups_id = cups_obj.create({'name': cups_name, 'id_municipi': municipi_ids[0]})
            raise Exception('Error: el cups {} no existe. Manual Create...'.format(cups_name))
        else:
            cups_id = cups_ids[0]

        # crear instalacion_re
        instalacion_re_id = instalacion_re_obj.create(
            {
                'name': instalacion_re_data['name'],
                'cups': cups_id,
                'titular': titular_id,
                'provincia': instalacion_re_data['provincia'],
                'tipo': instalacion_re_data['tipo_punto_fontera'],
                'potencia_nominal': instalacion_re_data['potencia_nominal']
            }
        )

        # UPRS
        representantes_ids = partner_obj.search([('name', '=', instalacion_re_data['representante'])])
        if not representantes_ids:
            representantes_ids = partner_obj.create(
                {
                    'name': instalacion_re_data['representante'], 
                    'ref': instalacion_re_data['cod_participante'],
                    'vat': 'ES' + instalacion_re_data['CIF'],
                }
            )
            representantes_ids = representantes_ids.id
            p_address_id = partner_addr.create({'name': instalacion_re_data['representante'], 'partner_id': representantes_ids})
            print('Representante creado. Validar datos: {}'.format(instalacion_re_data['representante']))
        else:
            representantes_ids = representantes_ids[0]
            ref = partner_obj.read(representantes_ids, ['ref'])['ref']
            if not ref:
                partner_obj.write(representantes_ids, {'ref': instalacion_re_data['cod_participante']})
        unidad_programacion_obj.create(
            {
                'name': instalacion_re_data['name_upr'], 
                'representante': representantes_ids, 
                'inst_re': instalacion_re_id.id,
                'data_alta': instalacion_re_data['data_alta'],
                'data_baixa': instalacion_re_data['data_baixa'],
            }
        )

In [140]:
df_instalaciones_re

,CIF,representante,name_upr,cod_participante,cups,data_alta,data_baixa,name,tipo_punto_fontera,unidad_lec,perdidas,p_ae,p_as,active,provincia
0,A62332580,"NEXUS ENERGIA, S.A",FYNEXU,0409,ES0034111000073020KN1F,2009-01-01,3000-01-01 00:00:00,"SOCIEDAD GRAL BAHIAS DE CADIZ Y ALGECIRAS, SA",3,1,0.0,False,False,True,CA
1,B82846825,"ENERGÍA XXI COMERCIALIZADORA DE REFERENCIA, S.L",FYEEXXI,0637,ES0034111302831001YT1F,2009-11-01,3000-01-01 00:00:00,COLEGIO OFICIAL INGL INDS. DE ANDALUCIA OCCIDE...,5,1,0.0,False,False,True,CA
2,B82846825,"ENERGÍA XXI COMERCIALIZADORA DE REFERENCIA, S.L",FYEEXXI,0637,ES0034111000073842BY1F,2009-11-01,3000-01-01 00:00:00,HARINERA VILAFRANQUINA - ZONA FRANCA,3,1,0.0,False,False,True,CA
3,B82846825,"ENERGÍA XXI COMERCIALIZADORA DE REFERENCIA, S.L",FYEEXXI,0637,ES0034111000073892JX1F,2009-11-01,3000-01-01 00:00:00,FUNDACION EE.PP SAFA CADIZ,3,1,0.0,False,False,True,CA
4,A84337849,GETERNOVA S.A,FYGESTE,0607,ES0034111000073893JB1F,2008-12-06,3000-01-01 00:00:00,SOL HERMES NEO,3,1,0.0,False,False,True,CA
5,A84337849,GETERNOVA S.A,FYGESTE,0607,ES0034111000073894JN1F,2008-12-06,3000-01-01 00:00:00,RENOVA ENERGIA SOLAR,3,1,0.0,False,False,True,CA
6,B82846825,"ENERGÍA XXI COMERCIALIZADORA DE REFERENCIA, S.L",FYEEXXI,0637,ES0034111000074891XC1F,2009-11-01,3000-01-01 00:00:00,CUBIERTA FOTOVOLTAICA PARQUE DE BOMBEROS DE CADIZ,3,1,0.0,False,False,True,CA
7,A62332580,"NEXUS ENERGIA, S.A",FYNEXU,0409,ES0034111000079283VL1F,2009-01-01,3000-01-01 00:00:00,FACHADA FOTOVOLTAICA APARCAMIENTO RECINTO INTE...,3,1,0.0,False,False,True,CA
8,A62332580,"NEXUS ENERGIA, S.A",FYNEXU,0409,ES0034111000079284VC1F,2009-01-01,3000-01-01 00:00:00,CUBIERTA FOTOVOLTAICA NAVE CITA,3,1,0.0,False,False,True,CA
9,B82846825,"ENERGÍA XXI COMERCIALIZADORA DE REFERENCIA, S.L",FYEEXXI,0637,ES0034111000080143DM1F,2009-11-01,3000-01-01 00:00:00,INSTALACION FOTOVOLTAICA SILICIO AMORFO,5,1,0.0,False,False,True,CA
